### Loading libs

In [ ]:
### Testing simple mesh substraction

import os
import sys
import math
os.environ["_CCTRACE_"]="ON"
import gendata as gen
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import laspy
import cloudComPy as cc 
import psutil

print("Libs loaded")

### Initialising cloud

In [ ]:
# os.environ["_CCTRACE_"]="ON"                                           # only if you want debug traces from C++

cc.initCC()
gen.createSymbolicLinks() # required for tests on build, before cc.initCC.init

cloud = cc.loadPointCloud(gen.getSamplePoly2(polyname="WhatTheHeck"))

### Creating primitives

In [ ]:
# ccGLMatrix() is a transformation matrix?
tr1 = cc.ccGLMatrix()
tr1.initFromParameters(0.1, 0.2, 0.3, (8.0, 0.0, 0.0))
box = cc.ccBox((1., 2., 3.), tr1, "aBox")
if box.getName() != 'aBox':
    raise RuntimeError
if box.size() != 12:
    raise RuntimeError

tr5 = cc.ccGLMatrix()
tr5.initFromParameters(0.0, (0., 0., 0.), (-7.0, 5.0, 1.0))
sphere = cc.ccSphere(1.5, tr5, "aSphere", 72)
if sphere.getName() != 'aSphere':
    raise RuntimeError
if sphere.size() != 10224:
    raise RuntimeError

### Computing approximate Cloud distance to mesh
[]: # Language: python
[]: # Path: testingFunctions.ipynb

In [ ]:
# stats = cc.DistanceComputationTools.computeApproxCloud2MeshDistance(cloud, sphere)
# print(stats) # min, max, mean, variance, error max
# nbCpu = psutil.cpu_count()
# bestOctreeLevel = cc.DistanceComputationTools.determineBestOctreeLevel(cloud,box)
# params = cc.Cloud2MeshDistancesComputationParams()
# params.maxThreadCount = nbCpu
# params.octreeLevel = bestOctreeLevel
# cc.DistanceComputationTools.computeCloud2MeshDistances(cloud, box, params)
# bestOctreeLevel = cc.DistanceComputationTools.determineBestOctreeLevel(box.getAssociatedCloud(), sphere)
# params.octreeLevel = bestOctreeLevel
# cc.DistanceComputationTools.computeCloud2MeshDistances(sphere.getAssociatedCloud(), sphere, params)

In [ ]:
cc.SaveEntities([cloud, box, sphere,], os.path.join("", "entities2.bin"))